In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import time
import os
import json
from base64 import b64decode
from io import BytesIO
from PIL import Image
from multiprocessing import Pool
from IPython.display import display

2023-02-24 20:25:10.759467: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 20:25:11.871026: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cudnn-8.2.4.15-11.4-eluwegpwn6adr7hlku5p5wru5xzefpop/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cuda-11.4.4-ctldo35wmmwws3jbgwkgjjcjawddu3qz/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/neovim-0.7.0-terkir3wk5rst6ktv4uxyaqjditacv5p/lib
2023-02-24 20:25:11.871105: W tensorflow/compiler/xla/stream_execut

### Constants

In [2]:
DATA_FOLDER = '/work/users/skoka/Data/WikipediaImageCaptionsProccessed/'

In [3]:
# lists all .json files in the data folder
files = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.json')]
files.sort()

In [4]:
# each json file has multiple json documents so we need to split them before loading
def load_file_into_json(file):
    with open(file) as f:
        json_list = [json.loads(line) for line in f.read().splitlines()]
    return json_list

In [5]:
first_file = load_file_into_json(DATA_FOLDER + files[0])

In [6]:
# Chat GPT parrallelized this code:
def process_json_doc(json_doc):
    X_data = []
    y_data = []
    unlabeled_images = []

    # Creating image from base64 bytes
    image_bytes = json_doc['b64_bytes']
    image_decoded = b64decode(image_bytes)
    image = Image.open(BytesIO(image_decoded)).convert("RGB") # Opens image from the decoded bytes
    image = image.resize((250, 250)) # Resizes all images to 250x250 to remain consistent
    image = np.array(image) # Converts image to numpy array

    english_caption = ""
    for caption in json_doc['wit_features']:
        if caption["language"] == "en":
            try:
                english_caption = caption["caption_reference_description"]
                break
            except:
                pass
    if english_caption == "":
        # decode base64 image
        unlabeled_images.append(image)
    else:
        X_data.append(image)
        y_data.append(english_caption)

    return X_data, y_data, unlabeled_images

def process_data(json_list):
    X_data = []
    y_data = []
    unlabeled_images = []

    # Creating a pool of processes
    pool = Pool()

    # Processing each JSON document in parallel using the process_json_doc function
    results = pool.map(process_json_doc, json_list)
    
    # Merging the results from each process
    for result in results:
        X_data += result[0]
        y_data += result[1]
        unlabeled_images += result[2]

    # Converting the lists to numpy arrays
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    unlabeled_images = np.array(unlabeled_images)

    return X_data, y_data, unlabeled_images

X_data, y_data, _ = process_data(first_file)

/users/skoka/.venv/tensorflow/lib/python3.8/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/users/skoka/.venv/tensorflow/lib/python3.8/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [7]:
X_data.shape

(5839, 250, 250, 3)

In [8]:
# GPT GENERATED
def display_image_caption(image, caption):
    # Convert the image to a PIL image
    image = Image.fromarray(image)
    # Display the image
    display(image)
    # Display the caption
    print(caption)